<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #11: Análisis de sentimiento
`Fecha de entrega: Noviembre 21, 2020 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [1]:
# cargar librerias 

# Estos paquetes los usaremos para analizar y visualizar los resultados
import pandas as pd
import tweepy
import numpy as np
import re
import regex
import emoji

# Libreias Plotly
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)
import seaborn as sns

from collections import Counter

import urllib.parse
import urllib.request
import time
import hmac
import hashlib
import random
import base64
import json

# Este es un modulo local de Python que contiene las credenciales de acceso para Twitter
import tweepy

from googletrans import Translator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

from flair.models import TextClassifier
from flair.data import Sentence

from wordcloud import WordCloud
from wordcloud import ImageColorGenerator

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')
import PIL.Image
from IPython.display import display

C:\ProgramData\Anaconda3\lib\site-packages\torch\cuda\__init__.py:52: UserWarning:

CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)



###  `[25 pts]` Punto 1: Hacer análisis de sentimiento de los datos del anterior taller
- Columna con el puntaje numérico del sentimiento
- Pueden usar cualquiera de las librerias vistas en clase

### Repetimos el ejercio del taller anterior "taller10" parta obtener los datos de twiter 

In [2]:
# Leer llaves
keys = pd.read_csv("C:/Users/practicas/Documents/Claves Tuit.csv", header=None)
keys= dict(zip(keys[0],keys[1]))

In [3]:
# Autorizar a través API
auth = tweepy.OAuthHandler(keys['api_key'],keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True) # se autoregula para recolectarinformacion

In [4]:
# Cargar archivo ciudades lat.-ln. bogota
ciudades = pd.read_csv("C:/Users/practicas/Documents/co_small.csv")
ciudades.head()

,city,lat,lng,country,iso2,admin,capital,population,population_proper
0,Bogotá,4.649178,-74.062827,Colombia,CO,Bogotá,primary,7772000.0,6333661.0
1,Medellín,6.251840,-75.563591,Colombia,CO,Antioquia,admin,3297000.0,1999979.0
2,Cali,3.437222,-76.522500,Colombia,CO,Valle del Cauca,admin,2254000.0,2178836.0
3,Barranquilla,10.963889,-74.796389,Colombia,CO,Atlántico,admin,1798000.0,1244491.0
4,Bucaramanga,7.125393,-73.119804,Colombia,CO,Santander,admin,1009000.0,571820.0


In [5]:
tweets = tweepy.Cursor(api.search,
                       q=['Selección Colombia futbol -RT'], # RT no incluir los retuis
                       lang='es', # solo tui en español
                           geocode=f'4.649178,-74.062827,45mi', #direccion geografica  bogota cambia a la que se quiera
                       tweet_mode='extended', # para que no recorte los texto del tuit
                       include_rts=False).items(100) # limitar el numero de twitwes

In [6]:
t = []
for tweet in tweets:
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    
    t.append(temp)

In [7]:
data = pd.DataFrame(t)
data.columns = ['tweet', 'date']
data.head()

,tweet,date
0,En los 60-70s Colombia se enriqueció mucho del...,2020-11-19 03:54:38
1,¿A Ancelotti le gustaría dirigir a Colombia? L...,2020-11-19 03:47:51
2,¿Cuáles jugadores pelearon en el camerino de l...,2020-11-19 03:04:41
3,Sin rodeos: Ancelotti respondió si le gustaría...,2020-11-19 02:52:47
4,¡A los golpes! Hubo pelea en el vestuario de l...,2020-11-19 02:51:15


In [11]:
#quitar enlaces y menciones
#quitar emojis

import re

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweet']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
             
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag,"")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention,"")
        
    return text, links, hashtags, mentions

In [12]:
#traducir tweet a ingles

translator = Translator()

data['ing'] = data['tweet'].apply(lambda valor: translator.translate(valor, src='es', dest='en'))
data.head()

,tweet,date,ing
0,En los 60-70s Colombia se enriqueció mucho del...,2020-11-19 03:54:38,"Translated(src=es, dest=en, text=In the 60-70s..."
1,¿A Ancelotti le gustaría dirigir a Colombia? L...,2020-11-19 03:47:51,"Translated(src=es, dest=en, text=Would Ancelot..."
2,¿Cuáles jugadores pelearon en el camerino de l...,2020-11-19 03:04:41,"Translated(src=es, dest=en, text=Which players..."
3,Sin rodeos: Ancelotti respondió si le gustaría...,2020-11-19 02:52:47,"Translated(src=es, dest=en, text=Bluntly: Ance..."
4,¡A los golpes! Hubo pelea en el vestuario de l...,2020-11-19 02:51:15,"Translated(src=es, dest=en, text=To the blows!..."


In [13]:
data['ingles'] = data['ing'].apply(lambda valor: valor.text)
data.head()

,tweet,date,ing,ingles
0,En los 60-70s Colombia se enriqueció mucho del...,2020-11-19 03:54:38,"Translated(src=es, dest=en, text=In the 60-70s...","In the 60-70s, Colombia was greatly enriched b..."
1,¿A Ancelotti le gustaría dirigir a Colombia? L...,2020-11-19 03:47:51,"Translated(src=es, dest=en, text=Would Ancelot...",Would Ancelotti like to lead Colombia? We tell...
2,¿Cuáles jugadores pelearon en el camerino de l...,2020-11-19 03:04:41,"Translated(src=es, dest=en, text=Which players...",Which players fought in the dressing room of t...
3,Sin rodeos: Ancelotti respondió si le gustaría...,2020-11-19 02:52:47,"Translated(src=es, dest=en, text=Bluntly: Ance...",Bluntly: Ancelotti answered if he would like t...
4,¡A los golpes! Hubo pelea en el vestuario de l...,2020-11-19 02:51:15,"Translated(src=es, dest=en, text=To the blows!...",To the blows! There was a fight in the dressin...


## Librerias  para analisis de sentimientos vistas en clases
#### No es necesario limpiar la data

In [14]:
classifier = TextClassifier.load('sentiment')


def sent_flair(texto):
    sentence = Sentence(texto)
    classifier.predict(sentence)
    result = sentence.labels[0]
    label = result.value
    score = result.score
    if label == 'POSITIVE':
        return score
    if label == 'NEGATIVE':
        return -1 * score
    return score

 
data['sent_flair'] = data['tweet'].apply(lambda valor: sent_flair(valor))
data.head()

2020-11-18 23:01:59,563 loading file C:\Users\practicas\.flair\models\sentiment-en-mix-distillbert_3.1.pt


,tweet,date,ing,ingles,sent_flair
0,En los 60-70s Colombia se enriqueció mucho del...,2020-11-19 03:54:38,"Translated(src=es, dest=en, text=In the 60-70s...","In the 60-70s, Colombia was greatly enriched b...",0.966915
1,¿A Ancelotti le gustaría dirigir a Colombia? L...,2020-11-19 03:47:51,"Translated(src=es, dest=en, text=Would Ancelot...",Would Ancelotti like to lead Colombia? We tell...,-0.578781
2,¿Cuáles jugadores pelearon en el camerino de l...,2020-11-19 03:04:41,"Translated(src=es, dest=en, text=Which players...",Which players fought in the dressing room of t...,-0.638301
3,Sin rodeos: Ancelotti respondió si le gustaría...,2020-11-19 02:52:47,"Translated(src=es, dest=en, text=Bluntly: Ance...",Bluntly: Ancelotti answered if he would like t...,0.982045
4,¡A los golpes! Hubo pelea en el vestuario de l...,2020-11-19 02:51:15,"Translated(src=es, dest=en, text=To the blows!...",To the blows! There was a fight in the dressin...,0.950778


In [15]:
data.loc[4]['tweet']

'¡A los golpes! Hubo pelea en el vestuario de la Selección Colombia tras la goleada ante Ecuador. \n\n“los dos protagonistas de la pelea habrían sido James Rodríguez y Jefferson Lerma.”  https://t.co/VbnRsFRNov via @MarcaClaroCol'

## Columna con el puntaje numérico del sentimiento

In [16]:
data[['tweet', 'sent_flair']].sort_values('sent_flair')


,tweet,sent_flair
37,@AmLuiissa No sirve. Ya tuvo su oportunidad co...,-0.999991
9,¿Camerino roto? Versiones sobre peleas y divis...,-0.999960
74,Opinión impopular: El salario del cuerpo técni...,-0.999810
59,"Colombia sin ""orgullo ni honor"" tras humillant...",-0.999481
61,¿De que otras formas necesitamos ser humilllad...,-0.999271
...,...,...
86,"Los procesos en el fútbol son muy complejos, C...",0.997595
58,Humillante 6-1: la peor derrota de la Selecció...,0.998198
75,"El señor Carlos Antonio Vélez sabe mucho, pero...",0.998571
82,Queiroz es el mejor técnico que ha tenido la S...,0.998609


### [10 pts] Punto 2: ¿Cuál es el tuit más positivo y más negativo


In [18]:
#tuit Negativo se organiza en orden a-z
data.loc[37]['tweet']

'@AmLuiissa No sirve. Ya tuvo su oportunidad con la selección y no hizo nada. Tampoco con la de México. No mas reciclaje de técnicos en Colombia, porque por eso es que el fútbol interno colombiano es tan mediocre y poco competitivo. Perdedores permanentes en todos los torneos intls también.'

In [19]:
#tuit Positivo se organiza de la z-a
data.loc[77]['tweet']

'Algunos tienen esperanza en la gente de Colombia y otros tenemos esperanza en la Selección Colombia de Fútbol.  Ambas esperanzas sacadas de quién sabe dónde.'

### `[15 pts]` Punto 3: ¿Cuántos tuits positivos/negativos hay en sus datos?

In [20]:
data['resultado'] = " "
data.loc[data['sent_flair']>0, 'resultado'] =  'tweet Positivo'
data.loc[data['sent_flair']<0, 'resultado'] =  'tweet Negativo'
data.loc[data['sent_flair']==0, 'resultado'] = 'tweet Neutro'
data

,tweet,date,ing,ingles,sent_flair,resultado
0,En los 60-70s Colombia se enriqueció mucho del...,2020-11-19 03:54:38,"Translated(src=es, dest=en, text=In the 60-70s...","In the 60-70s, Colombia was greatly enriched b...",0.966915,tweet Positivo
1,¿A Ancelotti le gustaría dirigir a Colombia? L...,2020-11-19 03:47:51,"Translated(src=es, dest=en, text=Would Ancelot...",Would Ancelotti like to lead Colombia? We tell...,-0.578781,tweet Negativo
2,¿Cuáles jugadores pelearon en el camerino de l...,2020-11-19 03:04:41,"Translated(src=es, dest=en, text=Which players...",Which players fought in the dressing room of t...,-0.638301,tweet Negativo
3,Sin rodeos: Ancelotti respondió si le gustaría...,2020-11-19 02:52:47,"Translated(src=es, dest=en, text=Bluntly: Ance...",Bluntly: Ancelotti answered if he would like t...,0.982045,tweet Positivo
4,¡A los golpes! Hubo pelea en el vestuario de l...,2020-11-19 02:51:15,"Translated(src=es, dest=en, text=To the blows!...",To the blows! There was a fight in the dressin...,0.950778,tweet Positivo
...,...,...,...,...,...,...
95,@DanielSanin Vaya a la selección colombia de f...,2020-11-18 00:06:00,"Translated(src=es, dest=en, text=@DanielSanin ...","@DanielSanin Go to the Colombian soccer team, ...",0.934637,tweet Positivo
96,"Alfaro, de comentar partidos de la Selección C...",2020-11-18 00:05:01,"Translated(src=es, dest=en, text=Alfaro, from ...","Alfaro, from commenting on matches of the Colo...",0.838315,tweet Positivo
97,Colombia fue humillada en Quito: cayó goleada ...,2020-11-18 00:03:55,"Translated(src=es, dest=en, text=Colombia was ...",Colombia was humiliated in Quito: they fell 6-...,0.986257,tweet Positivo
98,Juan Guillermo Cuadrado se refirió a la golead...,2020-11-18 00:00:46,"Translated(src=es, dest=en, text=Juan Guillerm...",Juan Guillermo Cuadrado referred to the win ag...,0.952445,tweet Positivo


In [21]:
# Conteo por tipo de sentimiento 
data = pd.DataFrame(data.resultado.value_counts()).reset_index()
data.columns = ['sent_flair', 'resultado']
data

,sent_flair,resultado
0,tweet Positivo,66
1,tweet Negativo,34


###  `[BONUS 5pts]` : Diagrama de barras de la pregunta 3


In [22]:
%matplotlib inline
init_notebook_mode(connected=True)

trace = go.Bar(x=data.sent_flair.values,
               y=data.resultado.values)

layout = go.Layout(title='Tweets x sentimientos')


In [23]:
fig = go.Figure(data = [trace], layout=layout)
iplot(fig)


###  `[BONUS 7pts]` : Nubes de palabras por sentimiento